### Download Data

https://surfdrive.surf.nl/files/index.php/s/iiACJcCJyQK8WD5

and put the data somewhere where python can see it.

### Install Dependencies

In [ ]:
# if your are working locally do something like
!pip install numpy pandas matplotlib seaborn sklearn pickle

#### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import pickle as pkl
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.svm import LinearSVC
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.model_selection import cross_val_score, LeaveOneOut
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif

---

### Load and Investigate Data

In [ ]:
with open('../../hippocampus_data.pkl', 'rb') as f:
    all_betas = pkl.load(f)

all_betas['DG']

### Visualize Features of Some Trials

In [ ]:
plt.figure(figsize=(14, 6))
sns.heatmap(all_betas['DG'][['BarackObama1_pa_cong', 'GeorgeBush1_ga_cong']].T)
plt.title('Some Congruent Trials')
plt.tight_layout()
plt.show()

---

## Exercise

Visualize some incongruent trials...

In [ ]:
# <Your Code Goes Here>

---

# Decoding $Congruent$ vs. $Incongruent$ Trials

In [ ]:
# pick region
region = 'CA2'

# set some hyper-parameters
train_test_ratio = .75
nr_of_selected_features = 200
regularization_penalty = 1.
max_iter = 200

# pick a balanced number of samples from both conditions
dataset = pd.concat([all_betas[region].filter(regex='_cong').iloc[:, np.random.choice(range(144), 24, replace=False)], all_betas[region].filter(regex='_incong')], axis=1)
betas = dataset.values.T
conds = dataset.columns.map(lambda x: 1 if '_cong' in x else 0) # encode classes
betas, conds = shuffle(betas, conds)

# split into train/test 75%/25%
n_trials = dataset.shape[1]
train_indices = np.random.choice(range(n_trials), int(train_test_ratio*n_trials), replace=False)
betas_train = betas[train_indices,:]
conds_train = conds[train_indices]
test_indices = [trial for trial in np.arange(n_trials) if trial not in train_indices]
betas_test = betas[test_indices,:]
conds_test = conds[test_indices]

# 1. Data Preprocessing 
# <was already performed.>

# 2. Normalize Data
betas_train = (betas_train-betas_train.mean())/betas_train.std()
betas_test = (betas_test-betas_train.mean())/betas_train.std() # we need to normalize using the train set to prevent information leakage

# 3. Select Features
feature_selection = SelectKBest(f_classif, k=nr_of_selected_features)
feature_selection.fit_transform(betas_train, conds_train).shape
selected_features = feature_selection.get_support(indices=True)
betas_train_selected = betas_train[:, selected_features]
betas_test_selected = betas_test[:, selected_features] # note that we are using the statistics from the train set again to prevent information leakage

# 4. Select Model
decoder = LogisticRegression(C=regularization_penalty, max_iter=max_iter)

# 5. Train Model
decoder.fit(betas_train_selected, conds_train)

y_predicted = decoder.predict(betas_test_selected)

acc = (y_predicted==conds_test).sum()/conds_test.size

print(f'Accuracy of predicted (i.e. decoded) conditions: {acc:0.2f}.')

### Exercise

Copy and paste below the code in the above cell to wrap it a loop.

In [ ]:
# pick region
region = 'CA2'

# set some hyper-parameters
train_test_ratio = .75
nr_of_selected_features = 200
regularization_penalty = 1.
max_iter = 200

acc_list = []
for _ in range(100):

    # <your code goes here>
    
    acc_list.append(acc)

    # here we print the mean and std of our obtained accuracies
print(f'Mean accuracy of predicted conditions: {np.mean(acc_list):0.2f}+-{np.std(acc_list):0.4f}.')

Now we can estimate the confidence in our decoder.

### Exercise

**Copy the code again below. Now test the different regions and adjust the following parameters to see if you can get an improved decoding accuracy:**
- region (DG, SUB, CA1, CA2, CA3)
- train_test_ratio (.5, 2/3, 3/4, 5/6)
- nr_of_selected_features (10, 50, 100, 200, 300)
- regularization_penalty = (0.001, 0.01, 0.1, 1., 10.)
- max_iter (100, 200, 500, 1000)

In [ ]:
# <Your Code Goes Here>

---

### We have seen above that there seems to be some information related to the experiment in the representations within hippocampus. **Let's now make this splitting of the data more systematic using a machine learning pipeline and cross-validation.** We will try to decode PA vs. GA instead of congruent vs. incongruent.


# Decoding $PA$ vs. $GA$ Trials using a (LOO) Cross-Validation

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression())
])

accs = {
    'cong': [], 'cong+incong(stim)': [], 'cong+incong(pred)': [], 'cong+omis': []
}

for region, df_betas in all_betas.items():

    # prepare congruent trials
    df_cong = df_betas.filter(regex='_cong')
    X = df_cong.values.T
    y = df_cong.columns.map(lambda x: 0. if '_pa_' in x else 1.).values

    # add incongruent trials
    df_incong = df_betas.filter(regex='_incong')
    X_incong = np.vstack([X, df_incong.values.T])
    y_heard = np.concatenate([y, df_incong.columns.map(lambda x: 0. if '_pa_' in x else 1.).values])
    y_pred = np.concatenate([y, df_incong.columns.map(lambda x: 1. if '_pa_' in x else 0.).values])

    # add omitted trials
    df_omis = df_betas.filter(regex='_omis')
    X_omis = np.vstack([X, df_omis.values.T])
    y_omis = np.concatenate([y, df_omis.columns.map(lambda x: 0. if '_pa_' in x else 1.).values])
    

    accs['cong'].append(cross_val_score(pipeline, X, y, cv=LeaveOneOut()))
    accs['cong+incong(stim)'].append(cross_val_score(pipeline, X_incong, y_heard, cv=LeaveOneOut()))
    accs['cong+incong(pred)'].append(cross_val_score(pipeline, X_incong, y_pred, cv=LeaveOneOut()))
    accs['cong+omis'].append(cross_val_score(pipeline, X_omis, y_omis, cv=LeaveOneOut()))

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(16,4))

for idx, key in enumerate(accs.keys()):
    sns.barplot(accs[key], ax=ax[idx], capsize=.1)
    ax[idx].axhline(.5, c='k', linestyle='--')
    ax[idx].set_title(key)
    ax[idx].set_xticks(range(5), all_betas.keys())
    if idx==0: ax[idx].set_ylabel('acc')

plt.suptitle('PA vs. GA Classification Results', fontweight='bold', fontsize=20)
plt.tight_layout()
plt.show()


# Let's see if feature selection will change / improve our results

In [ ]:
feature_percentage = .1

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selection', SelectKBest(f_classif)),
    ('classifier', LogisticRegression())
])

accs = {
    'cong': [], 'cong+incong(stim)': [], 'cong+incong(pred)': [], 'cong+omis': []
}

for region, df_betas in all_betas.items():

    # prepare congruent trials
    df_cong = df_betas.filter(regex='_cong')
    X = df_cong.values.T
    y = df_cong.columns.map(lambda x: 0. if '_pa_' in x else 1.).values

    # add incongruent trials
    df_incong = df_betas.filter(regex='_incong')
    X_incong = np.vstack([X, df_incong.values.T])
    y_heard = np.concatenate([y, df_incong.columns.map(lambda x: 0. if '_pa_' in x else 1.).values])
    y_pred = np.concatenate([y, df_incong.columns.map(lambda x: 1. if '_pa_' in x else 0.).values])

    # add omitted trials
    df_omis = df_betas.filter(regex='_omis')
    X_omis = np.vstack([X, df_omis.values.T])
    y_omis = np.concatenate([y, df_omis.columns.map(lambda x: 0. if '_pa_' in x else 1.).values])
    
    pipeline.set_params(feature_selection__k=max(1,int(X.shape[1]*feature_percentage)))

    accs['cong'].append(cross_val_score(pipeline, X, y, cv=LeaveOneOut()))
    accs['cong+incong(stim)'].append(cross_val_score(pipeline, X_incong, y_heard, cv=LeaveOneOut()))
    accs['cong+incong(pred)'].append(cross_val_score(pipeline, X_incong, y_pred, cv=LeaveOneOut()))
    accs['cong+omis'].append(cross_val_score(pipeline, X_omis, y_omis, cv=LeaveOneOut()))

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(16,4))

for idx, key in enumerate(accs.keys()):
    sns.barplot(accs[key], ax=ax[idx], capsize=.1)
    ax[idx].axhline(.5, c='k', linestyle='--')
    ax[idx].set_title(key)
    ax[idx].set_xticks(range(5), all_betas.keys())
    if idx==0: ax[idx].set_ylabel('acc')

plt.suptitle('PA vs. GA Classification Results', fontweight='bold', fontsize=20)
plt.tight_layout()
plt.show()

---

# Let's now look at multivariate feature selection

In [ ]:
feature_percentage = .1

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('feature_selection', SelectFromModel(Lasso(), threshold='mean')),
    ('classifier', LogisticRegression())
])

accs = {
    'cong': [], 'cong+incong(stim)': [], 'cong+incong(pred)': [], 'cong+omis': []
}

for region, df_betas in all_betas.items():

    # prepare congruent trials
    df_cong = df_betas.filter(regex='_cong')
    X = df_cong.values.T
    y = df_cong.columns.map(lambda x: 0. if '_pa_' in x else 1.).values

    # add incongruent trials
    df_incong = df_betas.filter(regex='_incong')
    X_incong = np.vstack([X, df_incong.values.T])
    y_heard = np.concatenate([y, df_incong.columns.map(lambda x: 0. if '_pa_' in x else 1.).values])
    y_pred = np.concatenate([y, df_incong.columns.map(lambda x: 1. if '_pa_' in x else 0.).values])

    # add omitted trials
    df_omis = df_betas.filter(regex='_omis')
    X_omis = np.vstack([X, df_omis.values.T])
    y_omis = np.concatenate([y, df_omis.columns.map(lambda x: 0. if '_pa_' in x else 1.).values])
    
    accs['cong'].append(cross_val_score(pipeline, X, y, cv=LeaveOneOut()))
    accs['cong+incong(stim)'].append(cross_val_score(pipeline, X_incong, y_heard, cv=LeaveOneOut()))
    accs['cong+incong(pred)'].append(cross_val_score(pipeline, X_incong, y_pred, cv=LeaveOneOut()))
    accs['cong+omis'].append(cross_val_score(pipeline, X_omis, y_omis, cv=LeaveOneOut()))

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(16,4))

for idx, key in enumerate(accs.keys()):
    sns.barplot(accs[key], ax=ax[idx], capsize=.1)
    ax[idx].axhline(.5, c='k', linestyle='--')
    ax[idx].set_title(key)
    ax[idx].set_xticks(range(5), all_betas.keys())
    if idx==0: ax[idx].set_ylabel('acc')

plt.suptitle('PA vs. GA Classification Results', fontweight='bold', fontsize=20)
plt.tight_layout()
plt.show()

---